In [1]:
# see https://towardsdatascience.com/how-do-i-extract-nested-data-in-python-4e7bed37566a

In [2]:
import pandas as pd
import requests
import json
from config import api_key
from pandas import json_normalize
import os

In [3]:
# pretty print the JSON
from pprint import pprint

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
# use below code to see full text of the columns; by providing None as the 2nd parameter, 
# we are setting the max characters allowed before truncation to unlimited
pd.set_option("display.max_colwidth", None)

In [4]:
os.getcwd()


'C:\\Users\\Jenkir\\Desktop\\cherry-tree-lane-analytics\\INDUSTRIES\\Materials-GICsector-EDA'

In [5]:
os.chdir('C:\\Users\\Jenkir\\Desktop\\cherry-tree-lane-analytics\\INDUSTRIES\\Materials-GICsector-EDA')
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\cherry-tree-lane-analytics\\INDUSTRIES\\Materials-GICsector-EDA'

In [6]:
# see https://www.youtube.com/watch?v=r7t71t5993s  for how to add quotes and commas to values in Excel
# id numbers source: 
headers = {
    "Api-key": api_key
          }

ids = ["193281",
"192130",
"244885",
"193361",
"191905",
"197846",
"193607",
"242957",
"191623",
"192198"
      ]

api_data = []

for i in ids:
    details = requests.get("https://api.esganalytics.io/api/company/details/?company_id={}".format(i), headers=headers)
    
    if details.status_code == 200:
        api_data.append(details.json())
    else:
        print("Request to { failed}".format(i))
   
pprint(api_data)

[{'all_executives': [{'bonus': 0,
                      'dob': '1965-01-01',
                      'name': 'Mr. John A. Hayes',
                      'non_equity_incentives': 3902076,
                      'option_awards': 2800000,
                      'other_comp': 67359,
                      'pension_and_deferred': 73183,
                      'salary': 1299013,
                      'stock_awards': 2800014,
                      'title': 'Chairman, Pres & CEO',
                      'total_compensation': 10941645},
                     {'bonus': 0,
                      'dob': '1966-01-01',
                      'name': 'Mr. Mark J. Costa',
                      'non_equity_incentives': 1540625,
                      'option_awards': 2946429,
                      'other_comp': 373043,
                      'pension_and_deferred': 186226,
                      'salary': 1226110,
                      'stock_awards': 9646050,
                      'title': 'Chairman & CEO',
       

                 {'code': '3788',
                  'currency_code': 'HKD',
                  'cusip': '',
                  'exchange': 'Hong Kong Exchange',
                  'exchange_id': 31,
                  'isin': 'KYG2115G1055',
                  'securities_type': 'Common Stock'}]},
 {'all_executives': [{'bonus': 0,
                      'dob': '1965-01-01',
                      'name': 'Mr. John A. Hayes',
                      'non_equity_incentives': 3902076,
                      'option_awards': 2800000,
                      'other_comp': 67359,
                      'pension_and_deferred': 73183,
                      'salary': 1299013,
                      'stock_awards': 2800014,
                      'title': 'Chairman, Pres & CEO',
                      'total_compensation': 10941645},
                     {'bonus': 0,
                      'dob': '1966-01-01',
                      'name': 'Mr. Mark J. Costa',
                      'non_equity_incentives': 15406

                                           {'date': '2018-08-03',
                                            'esg_beta': None},
                                           {'date': '2018-08-06',
                                            'esg_beta': None},
                                           {'date': '2018-08-07',
                                            'esg_beta': None},
                                           {'date': '2018-08-08',
                                            'esg_beta': None},
                                           {'date': '2018-08-09',
                                            'esg_beta': None},
                                           {'date': '2018-08-10',
                                            'esg_beta': None},
                                           {'date': '2018-08-13',
                                            'esg_beta': None},
                                           {'date': '2018-08-15',
                               

                                            'esg_beta': -0.466417174},
                                           {'date': '2019-08-05',
                                            'esg_beta': -0.4965862777},
                                           {'date': '2019-08-06',
                                            'esg_beta': -0.4540327578},
                                           {'date': '2019-08-07',
                                            'esg_beta': -0.3337624176},
                                           {'date': '2019-08-08',
                                            'esg_beta': -0.1768768525},
                                           {'date': '2019-08-09',
                                            'esg_beta': 0.0162854889},
                                           {'date': '2019-08-12',
                                            'esg_beta': 0.0314331254},
                                           {'date': '2019-08-13',
                                     

                                           {'date': '2020-10-28',
                                            'esg_beta': None},
                                           {'date': '2020-10-29',
                                            'esg_beta': None},
                                           {'date': '2020-10-30',
                                            'esg_beta': None},
                                           {'date': '2020-11-02',
                                            'esg_beta': None},
                                           {'date': '2020-11-03',
                                            'esg_beta': None},
                                           {'date': '2020-11-04',
                                            'esg_beta': None},
                                           {'date': '2020-11-05',
                                            'esg_beta': None},
                                           {'date': '2020-11-06',
                               

                  'option_awards': 0,
                  'other_comp': 0,
                  'pension_and_deferred': 0,
                  'salary': 0,
                  'stock_awards': 0,
                  'title': 'Sec. to the Board & Joint Company Sec.',
                  'total_compensation': 0},
                 {'bonus': 0,
                  'dob': '1964-01-01',
                  'name': 'Ms. Meifeng  Gu',
                  'non_equity_incentives': 0,
                  'option_awards': 0,
                  'other_comp': 0,
                  'pension_and_deferred': 0,
                  'salary': 0,
                  'stock_awards': 0,
                  'title': 'Chief Accountant',
                  'total_compensation': 0}],
  'indicators': [],
  'securities': [{'code': 'CMCLF',
                  'currency_code': 'USD',
                  'cusip': '',
                  'exchange': ' Pink',
                  'exchange_id': 72,
                  'isin': '',
                  'securities

                  'isin': 'HK0000112026',
                  'securities_type': 'Common Stock'}]},
 {'all_executives': [{'bonus': 0,
                      'dob': '1965-01-01',
                      'name': 'Mr. John A. Hayes',
                      'non_equity_incentives': 3902076,
                      'option_awards': 2800000,
                      'other_comp': 67359,
                      'pension_and_deferred': 73183,
                      'salary': 1299013,
                      'stock_awards': 2800014,
                      'title': 'Chairman, Pres & CEO',
                      'total_compensation': 10941645},
                     {'bonus': 0,
                      'dob': '1966-01-01',
                      'name': 'Mr. Mark J. Costa',
                      'non_equity_incentives': 1540625,
                      'option_awards': 2946429,
                      'other_comp': 373043,
                      'pension_and_deferred': 186226,
                      'salary': 1226110,
     

In [7]:
json_data = json.dumps(api_data)
dict_data = json.loads(json_data)
pprint(dict_data)

[{'all_executives': [{'bonus': 0,
                      'dob': '1965-01-01',
                      'name': 'Mr. John A. Hayes',
                      'non_equity_incentives': 3902076,
                      'option_awards': 2800000,
                      'other_comp': 67359,
                      'pension_and_deferred': 73183,
                      'salary': 1299013,
                      'stock_awards': 2800014,
                      'title': 'Chairman, Pres & CEO',
                      'total_compensation': 10941645},
                     {'bonus': 0,
                      'dob': '1966-01-01',
                      'name': 'Mr. Mark J. Costa',
                      'non_equity_incentives': 1540625,
                      'option_awards': 2946429,
                      'other_comp': 373043,
                      'pension_and_deferred': 186226,
                      'salary': 1226110,
                      'stock_awards': 9646050,
                      'title': 'Chairman & CEO',
       

                  'pension_and_deferred': 0,
                  'salary': 0,
                  'stock_awards': 0,
                  'title': 'Exec. Director',
                  'total_compensation': 0},
                 {'bonus': 0,
                  'dob': '1971-01-01',
                  'name': 'Mr. Yan  Wang',
                  'non_equity_incentives': 0,
                  'option_awards': 0,
                  'other_comp': 0,
                  'pension_and_deferred': 0,
                  'salary': 0,
                  'stock_awards': 0,
                  'title': 'Chairman',
                  'total_compensation': 0},
                 {'bonus': 0,
                  'dob': '1975-01-01',
                  'name': 'Mr. Zhimiao  Chen',
                  'non_equity_incentives': 0,
                  'option_awards': 0,
                  'other_comp': 0,
                  'pension_and_deferred': 0,
                  'salary': 0,
                  'stock_awards': 0,
                  'titl

                      'dob': '1962-01-01',
                      'name': 'Mr. Edward L. Doheny II',
                      'non_equity_incentives': 0,
                      'option_awards': 0,
                      'other_comp': 59118,
                      'pension_and_deferred': 0,
                      'salary': 1150000,
                      'stock_awards': 7725109,
                      'title': 'CEO, Pres & Director',
                      'total_compensation': 8934227},
                     {'bonus': 0,
                      'dob': '1962-01-01',
                      'name': 'Mr. Andreas  Fibig',
                      'non_equity_incentives': 2634583,
                      'option_awards': 0,
                      'other_comp': 491396,
                      'pension_and_deferred': 0,
                      'salary': 1300000,
                      'stock_awards': 4095961,
                      'title': 'Chairman & CEO',
                      'total_compensation': 8521940},
        

                                     'It is also involved in the refining and '
                                     'sale of mineral products; purchase and '
                                     'sale of molybdenum and tungsten '
                                     'products; import and export of goods and '
                                     'technology; and hotel management, '
                                     'consulting, enterprise operating and '
                                     'management, asset management, logistics, '
                                     'and transportation businesses. The '
                                     'company has operations in China, '
                                     'Australia, Brazil and the Democratic '
                                     'Republic of the Congo, Switzerland, and '
                                     'internationally. China Molybdenum Co., '
                                     'Ltd. was founded in 2006 and is ba

                                           {'date': '2018-08-13',
                                            'esg_beta': None},
                                           {'date': '2018-08-15',
                                            'esg_beta': None},
                                           {'date': '2018-08-16',
                                            'esg_beta': None},
                                           {'date': '2018-08-17',
                                            'esg_beta': None},
                                           {'date': '2018-08-20',
                                            'esg_beta': None},
                                           {'date': '2018-08-21',
                                            'esg_beta': None},
                                           {'date': '2018-08-22',
                                            'esg_beta': None},
                                           {'date': '2018-08-23',
                               

...

                                            'esg_beta': None},
                                           {'date': '2021-03-01',
                                            'esg_beta': None},
                                           {'date': '2021-03-02',
                                            'esg_beta': None},
                                           {'date': '2021-03-03',
                                            'esg_beta': None},
                                           {'date': '2021-03-04',
                                            'esg_beta': None},
                                           {'date': '2021-03-05',
                                            'esg_beta': None},
                                           {'date': '2021-03-08',
                                            'esg_beta': None},
                                           {'date': '2021-03-09',
                                            'esg_beta': None},
                                  

                                           {'date': '2021-09-15',
                                            'esg_beta': None},
                                           {'date': '2021-09-16',
                                            'esg_beta': None},
                                           {'date': '2021-09-17',
                                            'esg_beta': None},
                                           {'date': '2021-09-20',
                                            'esg_beta': None},
                                           {'date': '2021-09-21',
                                            'esg_beta': None},
                                           {'date': '2021-09-22',
                                            'esg_beta': None},
                                           {'date': '2021-09-23',
                                            'esg_beta': None},
                                           {'date': '2021-09-24',
                               

                  'securities_type': 'Common Stock'},
                 {'code': '3993',
                  'currency_code': 'HKD',
                  'cusip': '',
                  'exchange': 'Hong Kong Exchange',
                  'exchange_id': 31,
                  'isin': 'CNE100000114',
                  'securities_type': 'Common Stock'},
                 {'code': '603993',
                  'currency_code': 'CNY',
                  'cusip': '',
                  'exchange': 'Shanghai Exchange',
                  'exchange_id': 46,
                  'isin': 'CNE100001NR0',
                  'securities_type': 'Common Stock'}]},
 {'all_executives': [{'bonus': 0,
                      'dob': '1965-01-01',
                      'name': 'Mr. John A. Hayes',
                      'non_equity_incentives': 3902076,
                      'option_awards': 2800000,
                      'other_comp': 67359,
                      'pension_and_deferred': 73183,
                      'salary':

                 {'bonus': 0,
                  'dob': '1963-01-01',
                  'name': 'Mr. You Rong Wang',
                  'non_equity_incentives': 0,
                  'option_awards': 0,
                  'other_comp': 0,
                  'pension_and_deferred': 0,
                  'salary': 0,
                  'stock_awards': 0,
                  'title': 'Director',
                  'total_compensation': 0}],
  'indicators': [],
  'securities': [{'code': 'CHNR',
                  'currency_code': 'USD',
                  'cusip': '169406105',
                  'exchange': ' NASDAQ Exchnage',
                  'exchange_id': 75,
                  'isin': 'VGG2110U1093',
                  'securities_type': 'Common Stock'}]},
 {'all_executives': [{'bonus': 0,
                      'dob': '1965-01-01',
                      'name': 'Mr. John A. Hayes',
                      'non_equity_incentives': 3902076,
                      'option_awards': 2800000,
               

                      'pension_and_deferred': 0,
                      'salary': 1150000,
                      'stock_awards': 7725109,
                      'title': 'CEO, Pres & Director',
                      'total_compensation': 8934227},
                     {'bonus': 0,
                      'dob': '1962-01-01',
                      'name': 'Mr. Andreas  Fibig',
                      'non_equity_incentives': 2634583,
                      'option_awards': 0,
                      'other_comp': 491396,
                      'pension_and_deferred': 0,
                      'salary': 1300000,
                      'stock_awards': 4095961,
                      'title': 'Chairman & CEO',
                      'total_compensation': 8521940},
                     {'bonus': 0,
                      'dob': '1959-01-01',
                      'name': 'Mr. Douglas M. Baker Jr.',
                      'non_equity_incentives': 2284000,
                      'option_awards': 5328337,
    

In [8]:
print(type(dict_data))


<class 'list'>


In [9]:
print(type(api_data))

<class 'list'>


In [10]:
print(len(dict_data))

10


In [11]:
print(len(api_data))

10


In [12]:
#pip install flatten_json

In [13]:
# see https://github.com/amirziai/flatten
#from flatten_json import flatten

In [14]:
#df_flatten = pd.DataFrame([flatten(dict_data)['id']])
#df_flatten

In [15]:
print(dict_data[0])

{'company_detail': [{'id': 193281, 'company_name': 'China BlueChemical Ltd', 'description': "China BlueChemical Ltd., together with its subsidiaries, develops, manufactures, and sells mineral fertilisers and chemical products in the People's Republic of China and internationally. The company operates through Urea, Phosphorus and Compound Fertiliser, Methanol, and Others segments. It offers urea; mono-ammonium phosphate, di-ammonium phosphate, and compound fertilizers; methanol; and bulk blending fertilizers, and polyformaldehyde and woven plastic bags. It is also involved in the trading of fertilizers and chemicals; mining, processing, manufacturing, and selling of phosphate; port operations; and provision of preparatory work for methanol and dimethyl-ether projects. In addition, the company offers transportation services; and overseas shipping services, as well as produces and supplies food-class carbon dioxide. The company was formerly known as CNOOC Chemical Limited and changed its 

In [16]:
# see https://stackoverflow.com/questions/47242845/pandas-json-normalize-with-very-nested-json


df_detail2 = pd.json_normalize(dict_data, 'company_detail')
df_detail2

id                                      company_name  \
0  193281                            China BlueChemical Ltd   
1  192130      China Daye Non-Ferrous Metals Mining Limited   
2  244885                       China Green Agriculture Inc   
3  193361                    China Hanking Holdings Limited   
4  191905                      China Hongqiao Group Limited   
5  197846                         China Molybdenum Co., Ltd   
6  193607  China National Building Material Company Limited   
7  242957                       China Natural Resources Inc   
8  191623       China Nonferrous Mining Corporation Limited   
9  192198              China Oriental Group Company Limited   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   description  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                               China BlueChemical Ltd., together with its subsidiaries, develops, manufactures, and sells mineral fertilisers and chemical products in the People's Republic of China and internationally. The company operates through Urea, Phosphorus and Compound Fertiliser, Methanol, and Others segments. It offers urea; mono-ammonium phosphate, di-ammonium phosphate, and compound fertilizers; methanol; and bulk blending fertilizers, and polyformaldehyde and woven plastic bags. It is also involved in the trading of fertilizers and chemicals; mining, processing, manufacturing, and selling of phosphate; port operations; and provision of preparatory work for methanol and dimethyl-ether projects. In addition, the company offers transportation services; and overseas shipping services, as well as produces and supplies food-class carbon dioxide. The company was formerly known as CNOOC Chemical Limited and changed its name to China BlueChemical Ltd. in April 2006. The company was founded in 2000 and is headquartered in Beijing, the People's Republic of China. China BlueChemical Ltd. is a subsidiary of China National Offshore Oil Corporation.   
1                                                                                                            

In [17]:
df_detail2.columns

Index(['id', 'company_name', 'description', 'gicSector', 'url', 'address',
       'gicSubIndustry', 'company_pulse', 'pos_neg_ratio', 'esg_beta',
       'esg_beta_tseries'],
      dtype='object')

In [18]:
# drop unnecessary columns
df_detail = df_detail2.drop(columns=['description', 'url', 'pos_neg_ratio', 'esg_beta_tseries'])
df_detail

,id,company_name,gicSector,address,gicSubIndustry,company_pulse,esg_beta
0,193281,China BlueChemical Ltd,Materials,"Kaikang CNOOC Mansion, Beijing, China, 100029",Fertilizers & Agricultural Chemicals,0.000000,NaN
1,192130,China Daye Non-Ferrous Metals Mining Limited,Materials,"Tower 3, China Hong Kong City, Kowloon, Hong Kong",Copper,0.000000,NaN
2,244885,China Green Agriculture Inc,Materials,"Borough A, Xi'an, China, 710065",Fertilizers & Agricultural Chemicals,-0.406725,NaN
3,193361,China Hanking Holdings Limited,Materials,"Hanking Tower, Shenyang, China, 110016",Steel,0.000000,NaN
4,191905,China Hongqiao Group Limited,Materials,"Huixian One Road, Zouping, China, 256200",Aluminum,0.169675,NaN
5,197846,"China Molybdenum Co., Ltd",Materials,"North of Yihe, Luoyang, China, 471500",Diversified Metals & Mining,0.073025,0.252665
6,193607,China National Building Material Company Limited,Materials,"Guohai Plaza, Tower 2, Beijing, China, 100036",Construction Materials,0.949450,NaN
7,242957,China Natural Resources Inc,Materials,"West Tower, Sheung Wan, Hong Kong",Diversified Metals & Mining,0.700606,NaN
8,191623,China Nonferrous Mining Corporation Limited,Materials,"CNMC Building, Beijing, China",Diversified Metals & Mining,0.000000,NaN
9,192198,China Oriental Group Company Limited,Materials,"Great Eagle Centre, Wan Chai, Hong Kong",Steel,0.000000,NaN


In [19]:
# sort by company name & will use later when joining with next df bc there are no common columns; use ignore_index=True to reset index & keep new index while discarding old one
df_detail_F = df_detail.sort_values(by=["company_name"],ignore_index=True)
df_detail_F 

,id,company_name,gicSector,address,gicSubIndustry,company_pulse,esg_beta
0,193281,China BlueChemical Ltd,Materials,"Kaikang CNOOC Mansion, Beijing, China, 100029",Fertilizers & Agricultural Chemicals,0.000000,NaN
1,192130,China Daye Non-Ferrous Metals Mining Limited,Materials,"Tower 3, China Hong Kong City, Kowloon, Hong Kong",Copper,0.000000,NaN
2,244885,China Green Agriculture Inc,Materials,"Borough A, Xi'an, China, 710065",Fertilizers & Agricultural Chemicals,-0.406725,NaN
3,193361,China Hanking Holdings Limited,Materials,"Hanking Tower, Shenyang, China, 110016",Steel,0.000000,NaN
4,191905,China Hongqiao Group Limited,Materials,"Huixian One Road, Zouping, China, 256200",Aluminum,0.169675,NaN
5,197846,"China Molybdenum Co., Ltd",Materials,"North of Yihe, Luoyang, China, 471500",Diversified Metals & Mining,0.073025,0.252665
6,193607,China National Building Material Company Limited,Materials,"Guohai Plaza, Tower 2, Beijing, China, 100036",Construction Materials,0.949450,NaN
7,242957,China Natural Resources Inc,Materials,"West Tower, Sheung Wan, Hong Kong",Diversified Metals & Mining,0.700606,NaN
8,191623,China Nonferrous Mining Corporation Limited,Materials,"CNMC Building, Beijing, China",Diversified Metals & Mining,0.000000,NaN
9,192198,China Oriental Group Company Limited,Materials,"Great Eagle Centre, Wan Chai, Hong Kong",Steel,0.000000,NaN


In [20]:
df_detail_F.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              10 non-null     int64  
 1   company_name    10 non-null     object 
 2   gicSector       10 non-null     object 
 3   address         10 non-null     object 
 4   gicSubIndustry  10 non-null     object 
 5   company_pulse   10 non-null     float64
 6   esg_beta        1 non-null      float64
dtypes: float64(2), int64(1), object(4)
memory usage: 688.0+ bytes


In [21]:
df_securities = pd.json_normalize(dict_data, 'securities')
df_securities

,exchange,exchange_id,code,isin,cusip,securities_type,currency_code
0,Pink,72,CBLUF,,,Common Stock,USD
1,Pink,72,CBLUY,,,Common Stock,USD
2,Frankfurt Exchange,12,H1Q,,,Common Stock,EUR
3,Hong Kong Exchange,31,3983,CNE1000002D0,,Common Stock,HKD
4,Frankfurt Exchange,12,TFTB,,,Common Stock,EUR
5,Hong Kong Exchange,31,0661,BMG2125C1029,,Common Stock,HKD
6,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD
7,Frankfurt Exchange,12,6CH,,,Common Stock,EUR
8,Hong Kong Exchange,31,3788,KYG2115G1055,,Common Stock,HKD
9,Hong Kong Exchange,31,1378,KYG211501005,,Common Stock,HKD


In [22]:
df_securities_US = df_securities.loc[df_securities['currency_code'] == "USD"]
df_securities_US

,exchange,exchange_id,code,isin,cusip,securities_type,currency_code
0,Pink,72,CBLUF,,,Common Stock,USD
1,Pink,72,CBLUY,,,Common Stock,USD
6,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD
10,Pink,72,CMCLF,,,Common Stock,USD
14,Pink,72,CBUMY,,16947K107,Common Stock,USD
17,NASDAQ Exchnage,75,CHNR,VGG2110U1093,169406105,Common Stock,USD
20,Pink,72,CUGCF,,,Common Stock,USD
21,Pink,72,CUGCY,,16948A108,Common Stock,USD


In [23]:
# select any duplicate rows 
dupes = df_securities_US[df_securities_US.duplicated("code")]
dupes


,exchange,exchange_id,code,isin,cusip,securities_type,currency_code


In [24]:
# drop duplicate rows
securities_final2= df_securities_US.drop_duplicates("code")
securities_final2

,exchange,exchange_id,code,isin,cusip,securities_type,currency_code
0,Pink,72,CBLUF,,,Common Stock,USD
1,Pink,72,CBLUY,,,Common Stock,USD
6,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD
10,Pink,72,CMCLF,,,Common Stock,USD
14,Pink,72,CBUMY,,16947K107,Common Stock,USD
17,NASDAQ Exchnage,75,CHNR,VGG2110U1093,169406105,Common Stock,USD
20,Pink,72,CUGCF,,,Common Stock,USD
21,Pink,72,CUGCY,,16948A108,Common Stock,USD


In [25]:
securities_final = securities_final2.sort_values(by=["code"], ignore_index=True)
securities_final

,exchange,exchange_id,code,isin,cusip,securities_type,currency_code
0,Pink,72,CBLUF,,,Common Stock,USD
1,Pink,72,CBLUY,,,Common Stock,USD
2,Pink,72,CBUMY,,16947K107,Common Stock,USD
3,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD
4,NASDAQ Exchnage,75,CHNR,VGG2110U1093,169406105,Common Stock,USD
5,Pink,72,CMCLF,,,Common Stock,USD
6,Pink,72,CUGCF,,,Common Stock,USD
7,Pink,72,CUGCY,,16948A108,Common Stock,USD


In [26]:
# merge the df that has company details with the df that has securities info; see https://www.statology.org/pandas-merge-on-index/

df_merged = pd.merge(df_detail_F, securities_final, left_index=True, right_index=True)
df_merged

,id,company_name,gicSector,address,gicSubIndustry,company_pulse,esg_beta,exchange,exchange_id,code,isin,cusip,securities_type,currency_code
0,193281,China BlueChemical Ltd,Materials,"Kaikang CNOOC Mansion, Beijing, China, 100029",Fertilizers & Agricultural Chemicals,0.000000,NaN,Pink,72,CBLUF,,,Common Stock,USD
1,192130,China Daye Non-Ferrous Metals Mining Limited,Materials,"Tower 3, China Hong Kong City, Kowloon, Hong Kong",Copper,0.000000,NaN,Pink,72,CBLUY,,,Common Stock,USD
2,244885,China Green Agriculture Inc,Materials,"Borough A, Xi'an, China, 710065",Fertilizers & Agricultural Chemicals,-0.406725,NaN,Pink,72,CBUMY,,16947K107,Common Stock,USD
3,193361,China Hanking Holdings Limited,Materials,"Hanking Tower, Shenyang, China, 110016",Steel,0.000000,NaN,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD
4,191905,China Hongqiao Group Limited,Materials,"Huixian One Road, Zouping, China, 256200",Aluminum,0.169675,NaN,NASDAQ Exchnage,75,CHNR,VGG2110U1093,169406105,Common Stock,USD
5,197846,"China Molybdenum Co., Ltd",Materials,"North of Yihe, Luoyang, China, 471500",Diversified Metals & Mining,0.073025,0.252665,Pink,72,CMCLF,,,Common Stock,USD
6,193607,China National Building Material Company Limited,Materials,"Guohai Plaza, Tower 2, Beijing, China, 100036",Construction Materials,0.949450,NaN,Pink,72,CUGCF,,,Common Stock,USD
7,242957,China Natural Resources Inc,Materials,"West Tower, Sheung Wan, Hong Kong",Diversified Metals & Mining,0.700606,NaN,Pink,72,CUGCY,,16948A108,Common Stock,USD


In [27]:
df_merged[["street_address", "City", "State", "Country", "zip"]] = df_merged["address"].str.split(",", expand=True)
df_merged

,id,company_name,gicSector,address,gicSubIndustry,company_pulse,esg_beta,exchange,exchange_id,code,isin,cusip,securities_type,currency_code,street_address,City,State,Country,zip
0,193281,China BlueChemical Ltd,Materials,"Kaikang CNOOC Mansion, Beijing, China, 100029",Fertilizers & Agricultural Chemicals,0.000000,NaN,Pink,72,CBLUF,,,Common Stock,USD,Kaikang CNOOC Mansion,Beijing,China,100029,None
1,192130,China Daye Non-Ferrous Metals Mining Limited,Materials,"Tower 3, China Hong Kong City, Kowloon, Hong Kong",Copper,0.000000,NaN,Pink,72,CBLUY,,,Common Stock,USD,Tower 3,China Hong Kong City,Kowloon,Hong Kong,None
2,244885,China Green Agriculture Inc,Materials,"Borough A, Xi'an, China, 710065",Fertilizers & Agricultural Chemicals,-0.406725,NaN,Pink,72,CBUMY,,16947K107,Common Stock,USD,Borough A,Xi'an,China,710065,None
3,193361,China Hanking Holdings Limited,Materials,"Hanking Tower, Shenyang, China, 110016",Steel,0.000000,NaN,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD,Hanking Tower,Shenyang,China,110016,None
4,191905,China Hongqiao Group Limited,Materials,"Huixian One Road, Zouping, China, 256200",Aluminum,0.169675,NaN,NASDAQ Exchnage,75,CHNR,VGG2110U1093,169406105,Common Stock,USD,Huixian One Road,Zouping,China,256200,None
5,197846,"China Molybdenum Co., Ltd",Materials,"North of Yihe, Luoyang, China, 471500",Diversified Metals & Mining,0.073025,0.252665,Pink,72,CMCLF,,,Common Stock,USD,North of Yihe,Luoyang,China,471500,None
6,193607,China National Building Material Company Limited,Materials,"Guohai Plaza, Tower 2, Beijing, China, 100036",Construction Materials,0.949450,NaN,Pink,72,CUGCF,,,Common Stock,USD,Guohai Plaza,Tower 2,Beijing,China,100036
7,242957,China Natural Resources Inc,Materials,"West Tower, Sheung Wan, Hong Kong",Diversified Metals & Mining,0.700606,NaN,Pink,72,CUGCY,,16948A108,Common Stock,USD,West Tower,Sheung Wan,Hong Kong,None,None


In [28]:
df_merged.columns

Index(['id', 'company_name', 'gicSector', 'address', 'gicSubIndustry',
       'company_pulse', 'esg_beta', 'exchange', 'exchange_id', 'code', 'isin',
       'cusip', 'securities_type', 'currency_code', 'street_address', 'City',
       'State', 'Country', 'zip'],
      dtype='object')

In [29]:
df_merged_temp = df_merged.rename(columns={'code': "symbol",
                                           'id': 'unique_id'
                                        })
df_merged_temp

,unique_id,company_name,gicSector,address,gicSubIndustry,company_pulse,esg_beta,exchange,exchange_id,symbol,isin,cusip,securities_type,currency_code,street_address,City,State,Country,zip
0,193281,China BlueChemical Ltd,Materials,"Kaikang CNOOC Mansion, Beijing, China, 100029",Fertilizers & Agricultural Chemicals,0.000000,NaN,Pink,72,CBLUF,,,Common Stock,USD,Kaikang CNOOC Mansion,Beijing,China,100029,None
1,192130,China Daye Non-Ferrous Metals Mining Limited,Materials,"Tower 3, China Hong Kong City, Kowloon, Hong Kong",Copper,0.000000,NaN,Pink,72,CBLUY,,,Common Stock,USD,Tower 3,China Hong Kong City,Kowloon,Hong Kong,None
2,244885,China Green Agriculture Inc,Materials,"Borough A, Xi'an, China, 710065",Fertilizers & Agricultural Chemicals,-0.406725,NaN,Pink,72,CBUMY,,16947K107,Common Stock,USD,Borough A,Xi'an,China,710065,None
3,193361,China Hanking Holdings Limited,Materials,"Hanking Tower, Shenyang, China, 110016",Steel,0.000000,NaN,New York Stock Exchange,69,CGA,US16943W2044,16943W204,Common Stock,USD,Hanking Tower,Shenyang,China,110016,None
4,191905,China Hongqiao Group Limited,Materials,"Huixian One Road, Zouping, China, 256200",Aluminum,0.169675,NaN,NASDAQ Exchnage,75,CHNR,VGG2110U1093,169406105,Common Stock,USD,Huixian One Road,Zouping,China,256200,None
5,197846,"China Molybdenum Co., Ltd",Materials,"North of Yihe, Luoyang, China, 471500",Diversified Metals & Mining,0.073025,0.252665,Pink,72,CMCLF,,,Common Stock,USD,North of Yihe,Luoyang,China,471500,None
6,193607,China National Building Material Company Limited,Materials,"Guohai Plaza, Tower 2, Beijing, China, 100036",Construction Materials,0.949450,NaN,Pink,72,CUGCF,,,Common Stock,USD,Guohai Plaza,Tower 2,Beijing,China,100036
7,242957,China Natural Resources Inc,Materials,"West Tower, Sheung Wan, Hong Kong",Diversified Metals & Mining,0.700606,NaN,Pink,72,CUGCY,,16948A108,Common Stock,USD,West Tower,Sheung Wan,Hong Kong,None,None


In [30]:
df_merged_temp.columns

Index(['unique_id', 'company_name', 'gicSector', 'address', 'gicSubIndustry',
       'company_pulse', 'esg_beta', 'exchange', 'exchange_id', 'symbol',
       'isin', 'cusip', 'securities_type', 'currency_code', 'street_address',
       'City', 'State', 'Country', 'zip'],
      dtype='object')

In [31]:
# reorder the columns so that they match other datasets
df_details_secur = df_merged_temp[['unique_id', 'company_name', 'gicSector', 'gicSubIndustry','company_pulse', 
        'esg_beta', 'City', 'State', 'Country','zip','exchange', 'symbol', 'isin',
       'cusip', 'securities_type', 'currency_code']]

df_details_secur

,unique_id,company_name,gicSector,gicSubIndustry,company_pulse,esg_beta,City,State,Country,zip,exchange,symbol,isin,cusip,securities_type,currency_code
0,193281,China BlueChemical Ltd,Materials,Fertilizers & Agricultural Chemicals,0.000000,NaN,Beijing,China,100029,None,Pink,CBLUF,,,Common Stock,USD
1,192130,China Daye Non-Ferrous Metals Mining Limited,Materials,Copper,0.000000,NaN,China Hong Kong City,Kowloon,Hong Kong,None,Pink,CBLUY,,,Common Stock,USD
2,244885,China Green Agriculture Inc,Materials,Fertilizers & Agricultural Chemicals,-0.406725,NaN,Xi'an,China,710065,None,Pink,CBUMY,,16947K107,Common Stock,USD
3,193361,China Hanking Holdings Limited,Materials,Steel,0.000000,NaN,Shenyang,China,110016,None,New York Stock Exchange,CGA,US16943W2044,16943W204,Common Stock,USD
4,191905,China Hongqiao Group Limited,Materials,Aluminum,0.169675,NaN,Zouping,China,256200,None,NASDAQ Exchnage,CHNR,VGG2110U1093,169406105,Common Stock,USD
5,197846,"China Molybdenum Co., Ltd",Materials,Diversified Metals & Mining,0.073025,0.252665,Luoyang,China,471500,None,Pink,CMCLF,,,Common Stock,USD
6,193607,China National Building Material Company Limited,Materials,Construction Materials,0.949450,NaN,Tower 2,Beijing,China,100036,Pink,CUGCF,,,Common Stock,USD
7,242957,China Natural Resources Inc,Materials,Diversified Metals & Mining,0.700606,NaN,Sheung Wan,Hong Kong,None,None,Pink,CUGCY,,16948A108,Common Stock,USD


In [32]:
df_details_secur.dtypes

unique_id            int64
company_name        object
gicSector           object
gicSubIndustry      object
company_pulse      float64
esg_beta           float64
City                object
State               object
Country             object
zip                 object
exchange            object
symbol              object
isin                object
cusip               object
securities_type     object
currency_code       object
dtype: object

In [33]:
convert = {"unique_id": str
          }
df_final = df_details_secur.astype(convert)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 7
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unique_id        8 non-null      object 
 1   company_name     8 non-null      object 
 2   gicSector        8 non-null      object 
 3   gicSubIndustry   8 non-null      object 
 4   company_pulse    8 non-null      float64
 5   esg_beta         1 non-null      float64
 6   City             8 non-null      object 
 7   State            8 non-null      object 
 8   Country          7 non-null      object 
 9   zip              1 non-null      object 
 10  exchange         8 non-null      object 
 11  symbol           8 non-null      object 
 12  isin             8 non-null      object 
 13  cusip            8 non-null      object 
 14  securities_type  8 non-null      object 
 15  currency_code    8 non-null      object 
dtypes: float64(2), object(14)
memory usage: 1.1+ KB


In [34]:
# Check to see how many unique company names there are in our results
df_final.company_name.nunique(dropna=True)

8

In [35]:
# Export file as a CSV, without the Pandas index, but with the header
df_final.to_csv("materials3.csv", index = False, header=True)